In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import  BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [3]:
X,y=make_classification(n_samples=10000,n_features=10,n_informative=3)

In [4]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [5]:
dt=DecisionTreeClassifier(random_state=42)
dt.fit(X_train,y_train)
y_pred=dt.predict(X_test)

print("Decision Tree accuracy",accuracy_score(y_test,y_pred))

Decision Tree accuracy 0.9305


## Bagging

In [7]:
bag=BaggingClassifier(
base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.5,
    bootstrap=True,
    random_state=42
)

In [8]:
bag.fit(X_train,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [9]:
y_pred=bag.predict(X_test)

In [10]:
accuracy_score(y_test,y_pred)

0.9565

In [11]:
bag.estimators_samples_[0].shape

(4000,)

In [12]:
bag.estimators_features_[0].shape

(10,)

### Bagging using SVM

In [14]:
bag=BaggingClassifier(
base_estimator=SVC(),
n_estimators=500,
max_samples=0.25,
bootstrap=True,
random_state=42)

In [15]:
bag.fit(X_train,y_train)
y_pred=bag.predict(X_test)
print("Bagging using SVM",accuracy_score(y_test,y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Bagging using SVM 0.916


### Pasting

In [16]:
bag=BaggingClassifier(
base_estimator=DecisionTreeClassifier(),
n_estimators=500,
max_samples=0.25,
bootstrap=False,
random_state=42,
verbose=1,
n_jobs=-1)

In [17]:
bag.fit(X_train,y_train)
y_pred=bag.predict(X_test)
print("Pasing classifier",accuracy_score(y_test,y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    6.4s remaining:    6.4s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    6.4s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Pasing classifier 0.953


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s finished


### Random Subspaces

In [18]:
bag=BaggingClassifier(
base_estimator=DecisionTreeClassifier(),
n_estimators=500,
bootstrap=False,
max_features=0.5,
bootstrap_features=True,
random_state=42)

In [19]:
bag.fit(X_train,y_train)
y_pred=bag.predict(X_test)
print("Random Subspaces classifier",accuracy_score(y_test,y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Random Subspaces classifier 0.951


In [20]:
bag.estimators_samples_[0].shape

(8000,)

In [21]:
bag.estimators_features_[0].shape

(5,)

## Random Patches

In [23]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [24]:
bag.fit(X_train,y_train)
y_pred=bag.predict(X_test)
print("Random Patches classifier",accuracy_score(y_test,y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Random Patches classifier 0.944


## OOB Score

In [25]:
bag=BaggingClassifier(
base_estimator=DecisionTreeClassifier(),
n_estimators=500,
max_samples=0.25,
bootstrap=True,
oob_score=True,
random_state=42)

In [26]:
bag.fit(X_train,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, oob_score=True, random_state=42)

In [27]:
bag.oob_score_

0.952625

In [28]:
y_pred=bag.predict(X_test)
print("Accuracy",accuracy_score(y_test,y_pred))

Accuracy 0.954


##   Bagging Tips
- Bagging generally gives better results than Pasting
- good results come around the 25% to 50% row sampling
- Random patches and subspaces should be used while dealing with high dimensional data
- to find the correct hyperaparameter values we can do GrdiSearchCV/RandomSaerchCV

## Applying GridSearchCv

In [29]:
from sklearn.model_selection import GridSearchCV

In [37]:
parameters = {
    'n_estimators': [50,100,500], 
    'max_samples': [0.1,0.4,0.7,1.0],
    'bootstrap' : [True,False],
    'max_features' : [0.1,0.4,0.7,1.0]
    }

In [38]:

search = GridSearchCV(BaggingClassifier(), parameters, cv=5)

In [39]:
search.fit(X_train,y_train)

KeyboardInterrupt: 

In [ ]:
search.best_params_
search.best_score_

In [ ]:
search.best_params_